<a href="https://colab.research.google.com/github/listakurniawati/Sentiment-Analysis-with-CNN-BiLSTM/blob/main/Edutech_Sentiment_Analysis_with_CNN_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inisialisasi**

In [9]:
pip install google-play-scraper

     |████████████████████████████████| 51kB 3.2MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-0.2.1-cp37-none-any.whl size=22199 sha256=ce4ec1e4feca7ba5b5ac6069d7c444393fc7e48c2930cb6734bd620e4063b272
  Stored in directory: /root/.cache/pip/wheels/9a/fe/59/aaed9c7885041da68908f5e45f2796bf77696ac83874afd02c
Successfully built google-play-scraper


In [131]:
import pandas as pd
from google_play_scraper import app, Sort, reviews_all
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras

# **Scraping Review RuangGuru**

In [30]:
ruangguru = app(
    'com.ruangguru.livestudents',
    lang='id', # defaults to ''
    country='id' # defaults to 'id'
)

print(ruangguru)

{'title': 'Ruangguru - Super App Belajar', 'description': 'Ruangguru merupakan bimbingan belajar online di Indonesia dengan solusi terlengkap hanya dalam satu aplikasi!\r\n\r\nMulai dari video belajar, latihan soal, les privat, tryout, semua dapat diakses melalui smartphone. Kontennya tersedia untuk berbagai jenjang mulai dari SD, SMP hingga SMA sesuai dengan kurikulum nasional dan dirancang khusus oleh pengajar terbaik dan berpengalaman.\r\n\r\n\r\n- ruangbelajar\r\nBimbel online untuk belajar di mana saja dan kapan saja dengan puluhan ribu video beranimasi, ratusan ribu latihan soal, dan rangkuman berinfografis yang dipersiapkan dan diajarkan oleh Master Teacher terbaik.\r\n\r\nFitur-fitur ruangbelajar: \r\n1) Video Belajar Beranimasi\r\nTonton puluhan ribu video belajar beranimasi yang diajarkan oleh Master Teacher terbaik. Video juga bisa di-download supaya kamu bisa tonton berulang-ulang tanpa kuota. \r\n\r\n2) Latihan Soal dan Pembahasan\r\nPertajam pemahamanmu lewat lebih dari r

In [92]:
from google_play_scraper import Sort, reviews_all

result_rg = reviews_all(
    'com.ruangguru.livestudents',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'id'
    country='id', # defaults to 'id'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    count=1000,
    filter_score_with=5 # defaults to None(means all score)
)

print(result_rg)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [93]:
type(result_rg)

list

In [94]:
df_rg = pd.DataFrame(result_rg)
df_rg

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHY5XJCb05adZhMms6tcIFYtloxHGamttMuXJf...,Joni Raditiya,https://play-lh.googleusercontent.com/a-/AOh14...,Dulu kalo gak salah rating ruangguru di playst...,5,44,6.15.0,2021-05-21 14:29:44,"Hai Joni, terima kasih sudah memberikan bintan...",2021-05-25 12:48:24
1,gp:AOqpTOGr9j6MSZ53_4c6VXvFeSQvkgHmcUmwEhGRXPs...,Muhammad Farel alhasbi,https://play-lh.googleusercontent.com/a/AATXAJ...,"App nya bagus and funny, vdeo yg beranimasi mm...",5,13,6.15.0,2021-05-13 12:49:40,"Hai Farel, terima kasih telah memberikan binta...",2021-04-20 04:16:12
2,gp:AOqpTOHtM7R79TjW93_tPFNVY6wyhkI5cDBcTY04Tl0...,Henni Situmorang,https://play-lh.googleusercontent.com/a/AATXAJ...,Saya pengguna ruang guru ingin menyampaikan ka...,5,8,6.15.0,2021-05-14 10:10:34,"Halo Henni, terima kasih untuk saran dan binta...",2021-05-15 03:49:06
3,gp:AOqpTOElszwyD3rUudKxoC6CppG-JAzqOJaNdupzdHp...,Fikram Masloman,https://play-lh.googleusercontent.com/a-/AOh14...,Pelayanan dalam memberikan informasi sangat me...,5,1,6.15.0,2021-05-21 09:41:42,Hai Fikram. Yeay! Senang sekali bila kamu puas...,2021-05-21 10:35:44
4,gp:AOqpTOGW62MchsyhsXnwpOtDUU2rAqKmdFEGp0dMeWD...,Ahmad Fathan Asadurrahman,https://play-lh.googleusercontent.com/a-/AOh14...,Aplikasinya bagus. Cocok banget untuk belajar....,5,1,6.15.0,2021-05-20 12:02:43,"Halo, Fathan! Yeay, terima kasih untuk bintang...",2021-05-20 12:07:56
...,...,...,...,...,...,...,...,...,...,...
17706,gp:AOqpTOEvrb7qsuJwDpzOQpLyn40rxIzDrJXNdpwarZR...,Maria Ulfa,https://play-lh.googleusercontent.com/a/AATXAJ...,"Saya mau tanya ke pihak RG,kenapa ngak ada pel...",5,71,4.7.7.rc,2019-11-02 09:24:07,"Halo Maria, terima kasih sudah memberi review....",2019-11-14 07:18:40
17707,gp:AOqpTOFkE-THXbuoK2lfyunbDvsixjULPMPRWeclxxd...,Ami Azzahro,https://play-lh.googleusercontent.com/a/AATXAJ...,terima kasih dengan ada ruang guru belajar men...,5,0,3.3.0,2019-02-03 23:01:59,"Hai Ami Azzahro,\nTerimakasih atas review dan ...",2019-03-12 08:15:19
17708,gp:AOqpTOE47h8YFETYrfXUQyc5q3QaN4QoUuNXh0H-FAF...,Anisa Desy,https://play-lh.googleusercontent.com/a/AATXAJ...,Makasih buat ruangguru berkat ruangguru saya j...,5,0,4.6.4,2019-09-22 05:51:20,Hai Anisa. Terima kasih sudah belajar mengguna...,2019-09-22 06:08:50
17709,gp:AOqpTOH2bscZG2sEUc6IAtOPXzdMXjJIPu1fdYNLC0o...,Ahmad Fiqhi Fadli,https://play-lh.googleusercontent.com/a-/AOh14...,Aplikasi belajar yang sangat keren. Menghadirk...,5,4,4.1.3,2019-04-23 14:45:08,"Mantap deh Ahmad, keren banget ulasanmu. Ruang...",2019-12-01 01:32:33


In [147]:
rg_df = df_rg.drop(columns=['reviewId', 'userName', 'userImage', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt'])
rg_dfs = df_rg.drop(columns=['reviewId', 'userName', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt'])
rg_df

,content
0,Dulu kalo gak salah rating ruangguru di playst...
1,"App nya bagus and funny, vdeo yg beranimasi mm..."
2,Saya pengguna ruang guru ingin menyampaikan ka...
3,Pelayanan dalam memberikan informasi sangat me...
4,Aplikasinya bagus. Cocok banget untuk belajar....
...,...
17706,"Saya mau tanya ke pihak RG,kenapa ngak ada pel..."
17707,terima kasih dengan ada ruang guru belajar men...
17708,Makasih buat ruangguru berkat ruangguru saya j...
17709,Aplikasi belajar yang sangat keren. Menghadirk...


In [145]:
rg_df.to_csv('/content/rg_review.csv')

In [148]:
rg_dfs.to_csv('/content/rg_reviewscore.csv')

# **Upload Data**

In [ ]:
review = pd.read_csv('')

# ***Pre-processing***

In [132]:
def clean_text(tweet):
    
    # Convert to lower case
    tweet = tweet.lower()
    # Clean www.* or https?://*
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    # Clean @username
    tweet = re.sub('@[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    
    # Clean per Words
    words = tweet.split()
    tokens=[]
    for ww in words:
        #split repeated word
        for w in re.split(r'[-/\s]\s*', ww):
            #replace two or more with two occurrences
            pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
            w = pattern.sub(r"\1\1", w)
            #strip punctuation
            w = w.strip('\'"?,.')
            #check if the word cosists of two or more alphabets
            val = re.search(r"^[a-zA-Z][a-zA-Z][a-zA-Z]*$", w)
            #add tokens
            if(w in stopwords or val is None):
                continue
            else:
                tokens.append(w.lower())
    
    tweet = " ".join(tokens)
    return tweet